<a href="https://colab.research.google.com/github/RxnAch/ProjectsOnDeepLearning/blob/main/Chest_X_Ray_Images(Pneumonia).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np